#### Q1. Getting the embeddings model

In [2]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [ ]:
len(v)

In [4]:
v = embedding_model.encode(user_question)

round(v[0],3)

0.078

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef',
 'question_vector': array([ 1.55808441e-02, -2.45603397e-02,  6.55574426e-02,  4.41504307e-02,
         8.81910622e-02, -2.61536632e-02, -2.68038325e-02,  6.40117675e-02,
        -2.01325919e-02,  1.58315971e-02,  1.42648406e-02, -2.26580538e-02,
         8.13052133e-02, -2.77105123e-02,  4.82684299e-02,  1.24280266e-02,
         6.43338487e-02, -4.93125319e-02

In [15]:

documents = [doc for doc in documents if doc['course'] in "machine-learning-zoomcamp"]

#### Q2. Creating the embeddings

Create a list embeddings

Iterate over each document

qa_text = f'{question} {text}'

compute the embedding for qa_text, append to embeddings

At the end, let X = np.array(embeddings) (import numpy as np)

What's the shape of X? (X.shape). Include the parantheses.



In [17]:
import numpy as np 
from tqdm.auto import tqdm

embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    vector = embedding_model.encode(qa_text)
    embeddings.append(vector)

embeddings = np.array(embeddings)

100%|██████████| 375/375 [01:06<00:00,  5.62it/s]


In [19]:
embeddings.shape

(375, 768)

In [18]:
# embeddings = []

# for doc in tqdm(documents):
#     question = doc['question']
#     text = doc['text']
#     qt = question + ' ' + text

#     doc['question_vector'] = embedding_model.encode(question)
#     doc['text_vector'] = embedding_model.encode(text)
#     doc['question_text_vector'] = embedding_model.encode(qt)
#     embeddings.append(doc)

#### Q3. Search

In [22]:
scores = embeddings.dot(v)

max(scores)

0.6506573

#### Q4. Hit-rate for our search engine

In [31]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')


#### Q5. Indexing with Elasticsearch

#### Q6. Hit-rate for Elasticsearch